# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1350]:
# Dependencies 
import pandas as pd
from pathlib import Path

# Define file paths
school_file = Path("Resources/schools_complete.csv")
student_file = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_file)
student_df = pd.read_csv(student_file)

# Combine the data into a single dataset 
complete_school_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
complete_school_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [1351]:
# Calculate the total number of unique schools
school_count = len(complete_school_df['school_name'].unique())
school_count

15

In [1352]:
# Calculate the total number of students
student_count = complete_school_df['Student ID'].count()
student_count

39170

In [1353]:
# Calculate the total budget
total_budget = school_df['budget'].sum()
total_budget

24649428

In [1354]:
# Calculate the average math and reading score
average_math_score = complete_school_df['math_score'].mean()
average_reading_score = complete_school_df['reading_score'].mean()

In [1355]:
# Calculate the percentage of students who passed math, reading, and both (greather than or equal to 70)
passing_math_count = complete_school_df[(complete_school_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

passing_reading_count = complete_school_df[(complete_school_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

overall_passing_count = complete_school_df[(complete_school_df["math_score"] >= 70) & (complete_school_df["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = overall_passing_count / float(student_count) * 100

In [1357]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame([{'Total Schools': school_count,
                     'Total Students': student_count,
                     'Total Budget': total_budget,
                     'Average Math Score': average_math_score,
                     'Average Reading Score': average_reading_score,
                     'Passing Math (%)': passing_math_percentage,
                     'Passing Reading (%)': passing_reading_percentage,
                     'Overall Passing (%)': overall_passing_rate
                     }])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["Passing Math (%)"] = district_summary["Passing Math (%)"].map("{:,.2f}".format)
district_summary["Passing Reading (%)"] = district_summary["Passing Reading (%)"].map("{:,.2f}".format)
district_summary["Overall Passing (%)"] = district_summary["Overall Passing (%)"].map("{:,.2f}".format)


# Display the DataFrame
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading (%),Overall Passing (%)
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [1358]:
# New dataframe grouped by school_name
school_summary_df = complete_school_df.groupby('school_name')
school_summary_df.head()

# Select school type
school_type = school_df.set_index('school_name')['type']


In [1359]:
# Calculate the total student count
per_school_counts = school_summary_df['Student ID'].count()

In [1360]:
# Calculate the total school budget and per capita spending
per_school_budget = school_df.set_index('school_name')['budget']
per_student_budget = per_school_budget / per_school_counts


In [1361]:
# Calculate the average test scores
per_school_math = complete_school_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = complete_school_df.groupby(["school_name"]).mean()["reading_score"]

In [1362]:
# Calculate the number of schools with math and reading scores of 70 or higher
school_passing_math = complete_school_df[complete_school_df['math_score'] >= 70].groupby('school_name')['Student ID'].count()/per_school_counts * 100
school_passing_reading = complete_school_df[complete_school_df['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/per_school_counts *100
overall_passing_rate = complete_school_df[
    (complete_school_df['reading_score'] >= 70) & (complete_school_df["math_score"] >= 70)].groupby('school_name')['Student ID'].count()/per_school_counts * 100

In [1363]:
# # Create a DataFrame called `per_school_summary` with columns for the calculations above.

per_school_summary = pd.DataFrame({
                    'School Type': school_type,
                    'Total Students': per_school_counts,
                    'Total School Budget': per_school_budget,
                    'Per Student Budget': per_student_budget,
                    'Average Math Score': per_school_math,
                    'Average Reading Score': per_school_reading,
                    '% Passing Math': school_passing_math,
                    '% Passing Reading':school_passing_reading,
                    '% Overall Passing': overall_passing_rate
})


# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [1364]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending= False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [1365]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending= True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [1366]:
# Separate the data by grade
ninth_graders = complete_school_df[(complete_school_df["grade"] == "9th")]
tenth_graders = complete_school_df[(complete_school_df["grade"] == "10th")]
eleventh_graders = complete_school_df[(complete_school_df["grade"] == "11th")]
twelfth_graders = complete_school_df[(complete_school_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby('school_name').mean()
tenth_graders_scores = tenth_graders.groupby('school_name').mean()
eleventh_graders_scores = eleventh_graders.groupby('school_name').mean()
twelfth_graders_scores = twelfth_graders.groupby('school_name').mean()

# Select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "9th":ninth_grade_math_scores.map("{:,.2f}".format),
    "10th":tenth_grader_math_scores.map("{:,.2f}".format),
    "11th":eleventh_grader_math_scores.map("{:,.2f}".format),
    "12th":twelfth_grader_math_scores.map("{:,.2f}".format)
})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

In [1367]:
# Separate the data by grade
ninth_graders = complete_school_df[(complete_school_df["grade"] == "9th")]
tenth_graders = complete_school_df[(complete_school_df["grade"] == "10th")]
eleventh_graders = complete_school_df[(complete_school_df["grade"] == "11th")]
twelfth_graders = complete_school_df[(complete_school_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby('school_name').mean()
tenth_graders_scores = tenth_graders.groupby('school_name').mean()
eleventh_graders_scores = eleventh_graders.groupby('school_name').mean()
twelfth_graders_scores = twelfth_graders.groupby('school_name').mean()

# select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th":ninth_grade_reading_scores.map("{:,.2f}".format),
    "10th":tenth_grader_reading_scores.map("{:,.2f}".format),
    "11th":eleventh_grader_reading_scores.map("{:,.2f}".format),
    "12th":twelfth_grader_reading_scores.map("{:,.2f}".format)
})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

In [1368]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [1369]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [1370]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace('$', '').astype(float)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=labels)

In [1372]:
# Calculate averages for the desired columns. 
spending_group = school_spending_df.groupby("Spending Ranges (Per Student)")

spending_math_scores = spending_group["Average Math Score"].mean()
spending_reading_scores = spending_group["Average Reading Score"].mean()
spending_passing_math = spending_group["% Passing Math"].mean()
spending_passing_reading = spending_group["% Passing Reading"].mean()
overall_passing_rate = spending_group["% Overall Passing"].mean()

In [1373]:
# Assemble into DataFrame
spending_summary_df = pd.DataFrame({'Average Math Score' : spending_math_scores,
                        'Average Reading Score': spending_reading_scores,
                        '% Passing Math': spending_passing_math,
                        '% Passing Reading': spending_passing_reading,
                        '% Overall Passing Rate': overall_passing_rate
                        })

# Display results
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [1374]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [1375]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
school_size_df = per_school_summary.copy()
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], size_bins, labels=labels)

In [1377]:
# Calculate averages for the desired columns. 
size_group = school_size_df.groupby("School Size")

size_math_scores = school_size_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = school_size_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = school_size_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = school_size_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = school_size_df.groupby(["School Size"]).mean()["% Overall Passing"]

size_math_scores = size_group["Average Math Score"].mean()
size_reading_scores = size_group["Average Reading Score"].mean()
size_passing_math = size_group["% Passing Math"].mean()
size_passing_reading = size_group["% Passing Reading"].mean()
overall_passing_rate = size_group["% Overall Passing"].mean()

In [1378]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({'Average Math Score' : size_math_scores,
                 'Average Reading Score': size_reading_scores,
                 '% Passing Math': size_passing_math,
                 '% Passing Reading': size_passing_reading,
                 '% Overall Passing': overall_passing_rate
                 })

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [ ]:
# # Group the per_school_summary DataFrame by "School Type" and average the results.

# by_type = per_school_summary.set_index('School Type').groupby('School Type').mean()

# Filter per_school_summary to school type
school_types = per_school_summary.loc[per_school_summary["School Type"].isin(["Charter", "District"])]

# Calculate summary by school type
by_type = school_types.groupby("School Type").mean()

type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = (type_passing_math + type_passing_reading) / 2




In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Average Math Score" : type_math_scores,
                 'Average Reading Score': type_reading_scores,
                 '% Passing Math': type_passing_math,
                 '% Passing Reading': type_passing_reading,
                 '% Overall Passing Rate': overall_passing_rate
                 })

# Display results
type_summary